# Charging Model

In [2]:
# 导入模块
import xpress as xp
import numpy as np
import pandas as pd

In [3]:
# Create a problem called myproblem

prob = xp.problem(name='charging')

In [4]:
# Set index
number_of_types = 3 # i: 1..3
types = range(number_of_types)

number_of_years = 5 # Time t: 1..4 0..4
years = range(number_of_years)
number_of_years2 = number_of_years 
number_of_grids = 434 # grid g: 1..434
grids = range(number_of_grids)


In [5]:
# Define our decision variable as a numpy array
# 设置变量
x1 = np.array([xp.var( name='x1_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x2 = np.array([xp.var( name='x2_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years2)
# x = x1 + x2
z = np.array([xp.var( name='z_{0}_{1}'.format(g+1,t),vartype = xp.binary)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

x2 = np.array([xp.var( name='x2_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)

y = np.array([xp.var( name='y_{0}_{1}'.format(g+1,t),vartype = xp.integer)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

prob.addVariable(x1,x2,x,z)

In [6]:
# read data from files
pot_char_poi = pd.read_excel('Project_data/Potential_charging_points.xlsx')
intere_poi = pd.read_excel('Project_data/Interest _points.xlsx')
demand = pd.read_excel('Project_data/Demand_data.xlsx')
char_poi =  pd.read_excel('Project_data/Charging_points.xlsx')


In [7]:
#  Coefficient
# 设置参数
c = np.array(types) # building cost for charging point of type i
c = [1,1,1] # 可以修改

r = np.array(types) # the reduced building cost for charging point of type i
r = [0.8,0.8,0.8] # 可以修改

L = np.array(types) # the lower bound of proportion for type i charging point
L = [0.3,0.3,0.3]

Mit = np.array(types) # the higher bound of proportion for type i charging point
Mit = [0.9,0.9,0.9]

U = np.array(types) # the Upper bound of proportion for type i charging point ????
U = [0.6,0.6,0.6]

B = np.array(grids) # 每个grid 只能放这么多个charger，虽然我不是很理解,不应该每个location限制charger，每个grid限制的chrger等于location*这个数吗。。
for i in B:
    i = 10 # 暂定

T = 10 # the maximal number of station point in the centre of city : 可以自己定

N = 10 # lower bound of interest point in the neighbourhood : 可以自己定

TS = 0.9 # satisfaction proportion of for the total demand : 可以自己定

Mg = 0.9 # satisfaction proportion of for the demand in gird g : 可以自己定

# demand in region grid g in year t

Pi1 = np.array(years) # the minimum demand type i can provide per year
Pi1 = [2000,4000,30000]
Pi = np.array(years) # the maximum demand type i can provide per year # 这个后面好像没用到..
Pi = [3500,5200,55000]

In [8]:
Dgt = np.array(demand[['Demand_0','Demand_1','Demand_2','Demand_3']])
print(Dgt.shape)

(434, 4)


In [9]:
# 得到grid对应的potential locations
# Pg: the number of potential locations in the grid square
pg = (pot_char_poi['grid number'].value_counts())
Pg = np.zeros(number_of_grids) 
print(pg)
for i in pg.index:
    Pg[i-1] = pg[i]

200    18
186    15
89     13
199    13
103    11
       ..
144     1
101     1
291     1
160     1
331     1
Name: grid number, Length: 98, dtype: int64


In [10]:
y = np.array([xp.var( name='y_{0}_{1}'.format(g+1,t),vartype = xp.integer)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

prob.addVariable(y)

In [11]:
# 把第一年前已经建好charger的加入变量中
# number of charging point of type i in potential location j which has been established in year 1
# trans_type = []
# for i in char_poi['Type']:
#     if i == 'Slow':
#         trans_type.append(3)
#     elif i == 'Fast':
#         trans_type.append(3)
#     elif i == 'Rapid':
#         trans_type.append(3)
# exit_charger = pd.DataFrame({'Type':trans_type,'Grid':char_poi['grid number']})
char_poi['Type'].replace('Slow',0,inplace = True)
char_poi['Type'].replace('Fast',1,inplace = True)
char_poi['Type'].replace('Rapid',2,inplace = True)
# print(trans_type)
print(char_poi['Type'])

for i in types:
    for g in grids:
        if g+1 in list(char_poi['grid number']):
            prob.addConstraint( x[i,g,0] == char_poi[(char_poi.Type == i)&(char_poi['grid number'] == g+1)].shape[0])
        else:
            prob.addConstraint(x[i,g,0] == 0)

# prob.addConstraint(x[i,g,0] == len(exit_charger[(exit_charger['Type']==i)&(exit_charger['Grid']==g+1)]) 
#                    for i in types for g in grids)

0     0
1     1
2     0
3     0
4     1
     ..
73    0
74    0
75    0
76    0
77    1
Name: Type, Length: 78, dtype: int64


In [12]:
M = 10000

In [13]:
# 每年新建的 = 未开始使用的location新建charger数量 + 已开始使用的新建charger数量
prob.addConstraint(x[i,g,t] == x1[i,g,t] + x2[i,g,t] for i in types for g in grids for t in years)        

# y和x的关系 y(g,t) = 过去每一年的x......
for t in years:
    if t >0:
        for g in grids:
            prob.addConstraint(y[g,t] == xp.Sum(x[i,g,t1] for i in types for t1 in years if t1<=t-1 ))
    else:
        for g in grids:
            prob.addConstraint(y[g,t] == 0 )
for t in years:
    if t>0:
        for g in grids:
            for i in types:
                prob.addConstraint(M*z[g,t]>=y[g,t]  for g in grids )
                prob.addConstraint(y[g,t] >= z[g,t] for g in grids )
                prob.addConstraint(x1[i,g,t] <= M*z[g,t] for g in grids )
                prob.addConstraint(x2[i,g,t] <= M*(1-z[g,t]) for g in grids )
# for g in grids:
#     prob.addConstraint(y[g,0] == xp.Sum(x[i,g,0] for i in types) + xp.Sum(x[i,g,1] for i in types))
#     prob.addConstraint(y[g,1] == y[g,0] + xp.Sum(x[i,g,2] for i in types))
#     prob.addConstraint(y[g,2] == y[g,1] + xp.Sum(x[i,g,3] for i in types))
#     prob.addConstraint(y[g,3] == y[g,2] + xp.Sum(x[i,g,4] for i in types))

# for g in grids:
#     t = 0
#     while t <= 3: 
#         prob.addConstraint(y[g,t] == xp.Sum(x[i,g,t1] for t1 in range(t+2) for i in types))
#         t = t + 1

In [14]:
# 大M 




In [15]:
# 每年设施比例(每类充电桩/所有充电桩的比例): 这里我看公式的角标感到十分混乱。。。。
for i in types:
    for t in years:
        if t>0:
            prob.addConstraint(L[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years if t1<=t) <= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years if t1 <=t))
            prob.addConstraint(Mit[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years if t1<=t) >= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years if t1<=t))


In [16]:
#中心区域数量限制
distance = demand['Distance from Centre'].copy()
distance.sort_values(inplace = True)
center = distance[distance <= 1000]
center_grid = np.array(center.index)
print(center_grid)

[214 213 228 227 200 199 215 212 229 226 201 198]


In [17]:
for t in years:
    if t >0:
        number_center = 0
        for g in center_grid:
            number_center += xp.Sum(x[i,g-1,t1] for i in types for t1 in years if t1<= t )
        prob.addConstraint(number_center <= 100)


In [18]:
B = Pg*5                          #需要求
print(B)


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 15. 20.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 15. 65. 30.
 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  5. 15. 55. 15. 10.  5. 20.  0.
  5.  0.  0.  0.  0.  0. 20. 25. 20.  0. 25.  0. 20.  0.  5.  0.  0.  0.
  0.  0. 15.  0.  0.  0.  0. 20.  0.  0.  0.  0.  0.  0.  0.  0. 10.  5.
 10. 25.  5.  5.  5.  0.  0.  0.  0.  0.  0.  5.  5.  0.  0.  5. 15.  5.
  0.  5.  0.  0.  0.  0.  0.  0. 20. 15.  0.  0. 30. 45.  5.  5.  0.  0.
  0.  0.  0. 10. 40. 75. 20. 10.  5.  0. 15. 15.  0.  0.  0.  0.  0. 10.
 65. 90. 15. 15. 10.  0. 10.  5.  0.  0.  0.  0.  0.  0. 35. 45. 25.  5.
 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 50. 25. 30.  0.  0. 10.  5.
  0.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0. 10. 20

In [19]:
# potential point location number < Bg # 我改了下，直接用y了
for g in grids:
    for t in years:
        prob.addConstraint(y[g,t] <= B[g])

In [20]:
# electricity demand 请问G‘怎么定义,我很迷茫，我暂时定义为他的邻居吧。。。。。。。。。。还有为什么突然有了个t‘，我很迷茫
# 这里的formulation文件有两个公式，我看着好像是一个意思？代码部分写的是第一个公式
neighbors = demand['NEIGHBORS']

for g in grids:
    neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
    for t in years:
        if t>0:
            prob.addConstraint(xp.Sum(Pi1[i]*x[i,n-1,t1] for i in types for n in neighbor for t1 in years if t1<=t) >= 0.8*Dgt[g,t-1])
for t in years:
    if t>0:
        prob.addConstraint(Pi1[i]*xp.Sum(x[i,g,t] for i in types for g in grids for t1 in years if t1<=t)>= 0.8*xp.Sum(Dgt[g,t-1] for g in grids))
        

In [22]:
# interest point
# 得到每个点interest point数
M2 = 5
M3 = 1000
IG = intere_poi['grid number'].value_counts()
IG2 = np.zeros(number_of_grids)
for i in IG.index:
    IG2[i-1] = IG[i]
IG2
for t in years:
    if t>0:
        for g in grids:
         neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
         for i in neighbor:
            prob.addConstraint(xp.Sum(IG2[g1-1] for g1 in neighbor) >= z[g,t]*M2)


    



In [23]:
# Objective function
prob.setObjective(xp.Sum(c[i]*x1[i,g,t] for i in types for g in grids for t in years) 
                  + xp.Sum(r[i] * x2[i,g,t] for i in types for g in grids for t in years),
                 sense = xp.minimize)


In [24]:
prob.write("problem","lp")

In [25]:
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 2:06:14, Nov 28, 2022
Heap usage: 2427MB (peak 2427MB, 1171MB system)
Minimizing MILP charging using up to 16 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
   9067832 rows        23870 cols     18382924 elements     23870 globals
 
 
The problem is infeasible due to row R8305
Presolve finished in 2 seconds
Heap usage: 1960MB (peak 3920MB, 1171MB system)
 *** Search completed ***
Problem is integer infeasible
  Solution time / primaldual integral :         2s/ 100.000000%
  Number of solutions found / nodes   :         0 /         0
